бейзлайн
- первая часть (нахождения новостей с упоминанием компаний)
    - использовать только заголовки (датасет https://github.com/yutkin/Lenta.Ru-News-Dataset)
    - ~находить компании в заголовках через заранее заготовленный список компаний с Московской биржи~
    - сначала попробовать только на Яндексе
- вторая часть (проверять, как конкретная новость повлияла на акции компании)
    - смотреть, изменилась ли цена (на 10%, Яндекс из-за закона Горелкина падал на 15%) на фиксированный процент в тот же день (далее можно попробовать 24 рабочих часа после новости)
    - таким образом сделаем датасет для второй задачи (вида "текст новости - компания - таргет (цена понизилась / осталась прежней / повысилась)")
    - обучить на трейне такого датасета модель, которая получает на вход эмбеддинг предложения и пытается предсказать влияние на акции (в какой-то степени это будет sentiment analysis), вначале можно простенькую модель из нескольких слоев с релу, в конце софтмакс на 3 класса



### Сейчас:
- ~найти, откуда взять данные по бирже~ -- yfinance
- ~посмотреть период закона Горелкина (27.07.19) (нужно проверить наличие тех же проблем, что внизу этой ячейки)~ -- вроде по графикам таких проблем нет
- ~сделать функцию, которая к каждой новости покажет изменение цены в процентах~
    - считать изменение к прошлому дню, если вчерашнее или сегодняшнее значение недоступно, то NaN
- ~засунуть эти значения к соответствующим новостям~
- ~посчитать таргеты (изменения вверх/вниз больше х% (1/-1) или 0)~
- добавить других компаний
    - где-то получить лист русских компаний, который есть в yahoo finance
- обучить модель (предобученную на sentiment analysis, добавить пару слоев сверху и выход из 3 логитов, дальше софтмакс и получаем вероятность класса)

### Проверки:
- что случилось после новостей о законе Горелкина

### Проблемы:
- ~Проблемы с данными от pandas_datareader, в примере ниже слишком часто акции торгуются в нуле
```mr = pandas_datareader.data.DataReader('YNDX', 'moex', start='2019-01-01', end='2019-10-31')
mr.plot(subplots=True, figsize=(5, 10))```~ -- больше не использую pandas datareader, использую yfinance
- ~из-за закона Горелкина (27.07.19) акции упали с 2520 до 2441, но без новостей падают еще больше (27.05.19, 2404 -> 2280)~ -- проблема из-за pandas datareader, больше не актуально

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tqdm
import dateutil
import datetime
import json
import os
import requests

- взять интерфакс, не только российские компании
- посмтреть глазами, насколько разумны -1 и 1 (и из-за этого возможно поменять параметры)
- достать данные за более короткий срок
- обучить классификатор, ненулевых таргетов нужно 1000 (для предобученных можно меньше)
- разница -15 минут и 4 часа, смотреть разницу между ними, если больше 5-10%, то ненулвой таргет (также можно сравнить минимум-максимум за этот промежуток)

In [0]:
companies_to_listing_names = {
    'Яндекс': 'YNDX',
#     'Роснефть': 'ojscy',
#     'Лукойл': 'lukoy',
# #     'Сбербанк': 'SBER.ME',
#     'Сбербанк': 'SBRCY',
# #     'Русал': 'RUAL.ME',
# #     'Русал': '0486.HK',
#     'Газпром': 'OGZPY',
# #     'ВТБ': 'VTBR.ME',
# #     'Магнит': 'MGNT.ME',
# #     'Магнит': 'MGNT.IL',
# #     'Транснефть': 'TRNFP.ME',
# #     'X5 Retail Group': 'FIVE.ME',
# #     'Сургутнефтегаз': 'SGGD.VI',
#     'Сургутнефтегаз': 'SGTPY',
    
    'Apple': 'AAPL',
    'Microsoft': 'MSFT',
    'Google': 'GOOG',
    'Facebook': 'FB',
    'Amazon': 'AMZN',
    'Netflix': 'NFLX',
    'Ford': 'F',
    'General Motors': 'GM',
    'Alphabet': 'GOOG',
    'Verizon': 'VZ',
    'General Electric': 'GE',
    'Bank of America': 'BAC',
    'Boeing': 'BA',
    'Citigroup': 'C',
    'Citibank': 'C',
    'Dell': 'DELL',
    'IBM': 'IBM',
    'Intel': 'INTC',
    'Procter & Gamble': 'PG',
    'Pepsi': 'PEP',
    'Coca-Cola': 'KO',
    'AT&T': 'T',
    'Johnson & Johnson': 'JNJ',
    'Disney': 'DIS',
    'Nvidia': 'NVDA',
}

listing_names_to_companies = {value: key for (key, value) in companies_to_listing_names.items()}

In [0]:
!rm -rf interfax.tar.gz
!rm -rf interfax_20160101_20191015.csv
!wget https://www.dropbox.com/s/9byysrc9gs8cvbx/interfax.tar.gz
!tar -xzvf interfax.tar.gz

--2020-03-27 11:48:57--  https://www.dropbox.com/s/9byysrc9gs8cvbx/interfax.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/9byysrc9gs8cvbx/interfax.tar.gz [following]
--2020-03-27 11:48:58--  https://www.dropbox.com/s/raw/9byysrc9gs8cvbx/interfax.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4207986edd391aaf48e4fc82a0.dl.dropboxusercontent.com/cd/0/inline/A0twfv5upr8iGJsVh_YQc2qTiBjmf3QypXfk0OsTV3QbJ6f2Yr9AyDGBhQiOlSkNJt1HmNFjp02rzS1ZzgdJRS4aKAM6qpLFzUssbBXCn7Txl2-ywJXMqVL-bgMFzgAo8qg/file# [following]
--2020-03-27 11:48:58--  https://uc4207986edd391aaf48e4fc82a0.dl.dropboxusercontent.com/cd/0/inline/A0twfv5upr8iGJsVh_YQc2qTiBjmf3QypXfk0OsTV3QbJ6f2Yr9AyDGBhQiOlSkNJt1HmNFjp02rzS1ZzgdJRS4aKAM6qpLFzUssbBXCn

In [0]:
interfax_news_table = pd.read_csv(
    'interfax_20160101_20191015.csv',
    na_values='-',
    escapechar='\\',
    index_col=False,
    usecols=['date', 'title', 'text']
)

In [0]:
interfax_news_table.head()

,date,title,text
0,2019-09-09 14:09:00,Верховный суд в октябре рассмотрит жалобу на п...,Москва. 9 сентября. INTERFAX.RU - Верховный су...
1,2019-09-10 14:12:00,Суд Петербурга отменил выдворение блогера-оппо...,Москва. 10 сентября. INTERFAX.RU - Петербургск...
2,2019-09-11 14:20:00,Евро опустился ниже 72 рублей впервые со 2 авг...,Москва. 11 сентября. INTERFAX.RU - Евро в ходе...
3,2019-09-12 15:30:00,"ЕЦБ снизил ставку по депозитам до минус 0,5%",Москва. 12 сентября. INTERFAX.RU - Европейский...
4,2019-09-13 13:40:00,Москва негативно отнеслась к словам Зеленского...,Москва. 13 сентября. INTERFAX.RU - В Кремле не...


In [0]:
# Extract date
interfax_news_table['date'] = pd.to_datetime(
    interfax_news_table['date'],
    format='%y-%m-%d %H:%M:%S',
    exact=False
)

In [0]:
# Keep only companies-related news
company_news_table_parts = []
for name in companies_to_listing_names.keys():
    company_news_table = interfax_news_table
    company_news_table = company_news_table[company_news_table.title.str.contains(name, case=False)]
    company_news_table['company'] = name
    company_news_table_parts.append(company_news_table)

interfax_dataset = pd.concat(company_news_table_parts).reset_index(drop=True)

interfax_dataset.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,date,title,text,company
0,2019-09-24 12:31:00,"""Яндекс"" вернулся в список самых быстрорастущи...","Москва. 24 сентября. INTERFAX.RU - ""Яндекс"" вп...",Яндекс
1,2019-09-24 13:55:00,"Техдиректор ""ВКонтакте"" присоединится к фудтех...",Москва. 24 сентября. INTERFAX.RU - Новым техни...,Яндекс
2,2019-09-12 01:51:00,"""Яндекс"" вложит 5 млрд рублей в подготовку IT-...","Москва. 12 сентября. INTERFAX.RU - ""Яндекс"" об...",Яндекс
3,2019-09-11 09:49:00,"Gett пожаловался в ФАС на сделку ""Яндекс.Такси...","Сделка усилит доминирование сервиса ""Яндекса"" ...",Яндекс
4,2016-01-15 14:24:00,"""Яндекс.переводчик"" освоил язык эльфов",Москва. 15 января. INTERFAX.RU - Ко дню рожден...,Яндекс


In [0]:
len(interfax_dataset)

1896

In [0]:
len(interfax_dataset[interfax_dataset['date'] > '2017-05-31'])

1226

In [0]:
!wget https://www.dropbox.com/s/jjn0e05sxonyt78/tiingo_stocks.zip
!unzip tiingo_stocks.zip

--2020-03-27 11:13:51--  https://www.dropbox.com/s/jjn0e05sxonyt78/tiingo_stocks.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/jjn0e05sxonyt78/tiingo_stocks.zip [following]
--2020-03-27 11:13:52--  https://www.dropbox.com/s/raw/jjn0e05sxonyt78/tiingo_stocks.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc89941cb13bf7e5e02ed21f9749.dl.dropboxusercontent.com/cd/0/inline/A0slHGMpvMmNkF01XI9qm9vpgARkmZjhpI4OjopHymfB28hwWo9P-X7wu33So6JzhhIbIMlmxvgLKEedC7pIsfrXs2gwvCET4VobOCLt8KMrP-2YogJg0kwHZGE_jI22I2E/file# [following]
--2020-03-27 11:13:52--  https://uc89941cb13bf7e5e02ed21f9749.dl.dropboxusercontent.com/cd/0/inline/A0slHGMpvMmNkF01XI9qm9vpgARkmZjhpI4OjopHymfB28hwWo9P-X7wu33So6JzhhIbIMlmxvgLKEedC7pIsfrXs2gwvCET4Vo

In [0]:
import os
dir_path = './tiingo_stocks/'
# company_to_json_df = {}

def add_price_change(dataset, companies_to_listing_names):
    
    def get_close_price(df, time, fill_method):
        return df.iloc[df.index.get_loc(time, method=fill_method)]['close']

    def get_price_change(row):
        utc_date = row['date'] - datetime.timedelta(hours=3)  # Get UTC+0
        listing_name = companies_to_listing_names[row['company']]
        df = company_to_json_df[listing_name].set_index('date')
        left_time_bound = utc_date - datetime.timedelta(hours=1)
        right_time_bound = utc_date + datetime.timedelta(hours=3)
        try:
            old_price = get_close_price(df, left_time_bound, 'ffill')
            new_price = get_close_price(df, right_time_bound, 'bfill')
        except KeyError:
            return np.nan
        return (new_price - old_price) / old_price * 100, left_time_bound, right_time_bound, old_price, new_price
               

    def download_data(company_name):
        url = f'https://api.tiingo.com/iex/{company_name}/prices'
        params = {
            'startDate': '2015-01-01',
            # 'endDate': '2017-12-12',
            'resampleFreq': '60min',
            'token': os.environ.get('TIINGO_TOKEN', None)
        }

        r = requests.get(url=url, params=params)
        r.raise_for_status()
        data = r.json()

        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, 'w') as f:
            f.write(json.dumps(data))

    company_to_json_df = {}
    for listing_name in companies_to_listing_names.values():
        filename = dir_path + f'{listing_name}.json'
        if not os.path.isfile(filename):
            download_data(listing_name)
        # load json data to df
        company_to_json_df[listing_name] = pd.read_json(filename)#.set_index('date')

    return dataset.apply(get_price_change, axis=1)

interfax_dataset_price_change = interfax_dataset
column = add_price_change(interfax_dataset_price_change, companies_to_listing_names).dropna()
interfax_dataset_price_change['price_change'] = column.apply(lambda x: x[0])
interfax_dataset_price_change['left_time'] = column.apply(lambda x: x[1])
interfax_dataset_price_change['right_time'] = column.apply(lambda x: x[2])
interfax_dataset_price_change['old_price'] = column.apply(lambda x: x[3])
interfax_dataset_price_change['new_price'] = column.apply(lambda x: x[4])
interfax_dataset_price_change = interfax_dataset_price_change.dropna()
interfax_dataset_price_change['price_change']

0      -1.366867
1      -1.366867
2       0.238790
3       0.680363
101    -0.883978
          ...   
1891    0.000000
1892    1.715981
1893    0.477177
1894    2.197275
1895    7.291252
Name: price_change, Length: 1219, dtype: float64

In [0]:
threshold = 4
# nonzero_samples = interfax_dataset_price_change[(abs(interfax_dataset_price_change['price_change']) > threshold) & (abs(interfax_dataset_price_change['price_change']) < (threshold+1))]
nonzero_samples = interfax_dataset_price_change[(abs(interfax_dataset_price_change['price_change']) > threshold)]
print(len(nonzero_samples))
nonzero_samples

70


,date,title,text,company,price_change,left_time,right_time,old_price,new_price
112,2017-07-13 14:48:00,Uber получит 3 из 7 мест в совете директоров с...,Москва. 13 июля. INTERFAX.RU - Представители U...,Яндекс,16.261204,2017-07-13 10:48:00,2017-07-13 14:48:00,27.335,31.780
113,2017-07-13 17:00:00,"Глава ФАС прокомментировал сделку ""Яндекса"" и ...","По словам Игоря Артемьева, теоретически она мо...",Яндекс,15.072252,2017-07-13 13:00:00,2017-07-13 17:00:00,27.335,31.455
114,2017-07-13 17:23:00,"Финдиректор ""Яндекса"" не исключил IPO объедине...","Москва. 13 июля. INTERFAX.RU - ""Яндекс"" не иск...",Яндекс,15.200293,2017-07-13 13:23:00,2017-07-13 17:23:00,27.335,31.490
115,2017-07-13 13:16:00,"""Яндекс.Такси"" присоединило российский бизнес ...","Uber и ""Яндекс"" инвестируют соответственно $22...",Яндекс,15.291751,2017-07-13 09:16:00,2017-07-13 13:16:00,27.335,31.515
116,2017-07-13 13:57:00,"Акции ""Яндекса"" подскочили на 17,5% на новостя...","Москва. 13 июля. INTERFAX.RU - Акции ""Яндекса""...",Яндекс,15.291751,2017-07-13 09:57:00,2017-07-13 13:57:00,27.335,31.515
...,...,...,...,...,...,...,...,...,...
1775,2017-10-27 09:32:00,Чистая прибыль Intel за III квартал подскочила...,Санта-Клара. 27 октября. ИНТЕРФАКС - Intel Cor...,Intel,5.199516,2017-10-27 05:32:00,2017-10-27 09:32:00,41.350,43.500
1777,2018-01-26 09:50:00,Intel отчиталась о рекордной выручке в IV квар...,Москва. 26 января. INTERFAX.RU - Intel Corp по...,Intel,9.623662,2018-01-26 05:50:00,2018-01-26 09:50:00,45.305,49.665
1849,2018-06-13 08:11:00,Суд разрешил слияние Time Warner и AT&T,Сделку стоимостью $80 млрд обещал заблокироват...,AT&T,-5.138282,2018-06-13 04:11:00,2018-06-13 08:11:00,34.350,32.585
1870,2017-08-09 11:49:00,Disney запустит сервисы потокового видео и отк...,"Соглашение, позволявшее Netflix транслировать ...",Disney,-5.005141,2017-08-09 07:49:00,2017-08-09 11:49:00,106.990,101.635


In [0]:
def calc_target(dataset, threshold=threshold):
    return \
        (dataset['price_change'] > threshold) * 1 + \
        (dataset['price_change'] < -threshold) * -1

interfax_dataset_target = interfax_dataset_price_change
interfax_dataset_target['target'] = calc_target(interfax_dataset_target)
interfax_dataset_target = interfax_dataset_target.drop(['price_change', 'date'], axis=1)

len(interfax_dataset_target[interfax_dataset_target['target'] != 0])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


70

In [0]:
interfax_dataset_target

,title,text,company,left_time,right_time,old_price,new_price,target
0,"""Яндекс"" вернулся в список самых быстрорастущи...","Москва. 24 сентября. INTERFAX.RU - ""Яндекс"" вп...",Яндекс,2019-09-24 08:31:00,2019-09-24 12:31:00,36.58,36.080,0
1,"Техдиректор ""ВКонтакте"" присоединится к фудтех...",Москва. 24 сентября. INTERFAX.RU - Новым техни...,Яндекс,2019-09-24 09:55:00,2019-09-24 13:55:00,36.58,36.080,0
2,"""Яндекс"" вложит 5 млрд рублей в подготовку IT-...","Москва. 12 сентября. INTERFAX.RU - ""Яндекс"" об...",Яндекс,2019-09-11 21:51:00,2019-09-12 01:51:00,37.69,37.780,0
3,"Gett пожаловался в ФАС на сделку ""Яндекс.Такси...","Сделка усилит доминирование сервиса ""Яндекса"" ...",Яндекс,2019-09-11 05:49:00,2019-09-11 09:49:00,37.48,37.735,0
101,"""Яндекс.Навигатор"" стал показывать освободивши...","Москва. 6 июня. INTERFAX.RU - Программа ""Яндек...",Яндекс,2017-06-06 09:30:00,2017-06-06 13:30:00,27.15,26.910,0
...,...,...,...,...,...,...,...,...
1891,Русалочку в новом фильме Disney сыграет чернок...,Москва. 4 июля. INTERFAX.RU - Американская акт...,Disney,2019-07-04 05:30:00,2019-07-04 09:30:00,142.91,142.910,0
1892,Экс-глава Disney в России займется коммуникаци...,Москва. 10 июля. INTERFAX.RU - Экс-глава Disne...,Disney,2019-07-10 12:21:00,2019-07-10 16:21:00,141.61,144.040,0
1893,Будущее Человека-паука во франшизе Marvel стал...,"Студии не смогли договориться, как делить приб...",Disney,2019-08-21 09:31:00,2019-08-21 13:31:00,135.17,135.815,0
1894,Показатели Nvidia рекордно выросли за счет спр...,Москва. 9 февраля. INTERFAX.RU - Американский ...,Nvidia,2018-02-09 06:52:00,2018-02-09 10:52:00,218.68,223.485,0


In [0]:
dataset = interfax_dataset_target

In [0]:
# import deeppavlov

# # model = deeppavlov.build_model(deeppavlov.configs., download=True)
# from deeppavlov.models.bert.bert_classifier import BertClassifierModel
# from deeppavlov.models.preprocessors.bert_preprocessor import BertPreprocessor

# BertPreprocessor(
#     vocab_file='',
#     do_lower_case=True,
    
# )
# BertClassifierModel()

In [0]:
# # !mkdir /home/student/large_files_data/tayga_none_fasttextcbow_300_10_2019
parent_dir = '/home/student/large_files_data/'
# embs_path = parent_dir + 'tayga_none_fasttextcbow_300_10_2019.zip'
# !wget http://vectors.nlpl.eu/repository/11/187.zip -O {embs_path}
# !unzip {embs_path}

In [0]:
!wget http://vectors.nlpl.eu/repository/20/187.zip

--2020-03-27 11:42:50--  http://vectors.nlpl.eu/repository/20/187.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2692389554 (2.5G) [application/zip]
Saving to: ‘187.zip’

187.zip             100%[===================>]   2.51G  23.0MB/s    in 1m 56s  

2020-03-27 11:44:47 (22.2 MB/s) - ‘187.zip’ saved [2692389554/2692389554]



In [0]:
!unzip 187.zip

Archive:  187.zip
  inflating: meta.json               
  inflating: model.model             
  inflating: model.model.vectors_ngrams.npy  
  inflating: model.model.vectors.npy  
  inflating: model.model.vectors_vocab.npy  
  inflating: README                  


In [0]:
from gensim.models import KeyedVectors
unzipped_embs_path = 'model.model'
fasttext_model = KeyedVectors.load(unzipped_embs_path)

In [0]:
!pip install -U gensim razdel

     |████████████████████████████████| 24.2MB 1.3MB/s 
Requirement already up-to-date: razdel in /usr/local/lib/python3.6/dist-packages (0.5.0)
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [0]:
from razdel import tokenize
import numpy as np

def get_text_embedding(model, phrase):
    embeddings = np.array(
        [model.get_vector(word.text) for word in tokenize(phrase)]
    )
    return np.mean(embeddings, axis=0)
    
get_text_embedding(fasttext_model, "В Москве нашли")

array([-3.19591351e-02,  2.12788299e-01,  2.00785160e-01, -2.96731979e-01,
        1.72918045e-03,  6.50605261e-02,  4.87852544e-01, -4.68649358e-01,
       -1.82104707e-01,  4.29839015e-01, -4.72085953e-01, -4.38363701e-01,
        4.49346304e-01,  1.07596993e-01, -3.52612019e-01, -2.59015352e-01,
       -7.13011697e-02,  2.88060218e-01, -1.02649927e-02,  1.83870748e-01,
       -3.18037599e-01,  7.20639676e-02, -2.55334020e-01,  2.26732358e-01,
       -2.66987920e-01, -2.57337749e-01, -6.11868501e-03, -1.94196656e-01,
       -1.42383412e-01,  3.02220192e-02,  3.43344927e-01,  8.17791894e-02,
       -9.62877944e-02,  1.13385677e-01, -2.99482435e-01,  6.17247820e-03,
        3.29816341e-01,  1.81995317e-01,  4.08083439e-01,  6.34947792e-02,
       -5.47910392e-01,  8.84852409e-01, -3.69716972e-01,  1.82050750e-01,
       -3.89125973e-01, -9.03919637e-02,  4.13005501e-01,  5.61686695e-01,
        1.28564671e-01,  5.20360887e-01,  4.44802046e-01,  1.91065431e-01,
       -5.12464106e-01, -

In [0]:
y = dataset['target'].to_numpy()
X = np.zeros((dataset.shape[0], fasttext_model.vector_size))
for i, title in enumerate(dataset['title']):
    X[i, :] = get_text_embedding(fasttext_model, title)

In [0]:
X.shape

(1219, 300)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=0,
    stratify=y  # to keep targets proportion in train and test
)

In [0]:
from sklearn.neural_network import MLPClassifier
from sklearn import metrics

clf = MLPClassifier()
clf.fit(X_train, y_train)

y_predicted = clf.predict(X_test)
print(metrics.classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

          -1       0.25      0.12      0.17         8
           0       0.95      0.98      0.97       230
           1       0.50      0.17      0.25         6

    accuracy                           0.93       244
   macro avg       0.57      0.42      0.46       244
weighted avg       0.92      0.93      0.92       244



/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [0]:
from sklearn.preprocessing import LabelBinarizer

y_train_onehot = LabelBinarizer().fit_transform(y_train)
y_test_onehot = LabelBinarizer().fit_transform(y_test)

In [0]:
import keras
import keras.layers as L

keras_model = keras.Sequential()
keras_model.add(L.Dense(64, activation='relu', input_dim=fasttext_model.vector_size))
keras_model.add(L.Dropout(0.5))
keras_model.add(L.Dense(64, activation='relu'))
keras_model.add(L.Dropout(0.5))
keras_model.add(L.Dense(3, activation='softmax'))

sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
keras_model.compile(loss='categorical_crossentropy',
                    optimizer=sgd,
                    metrics=['accuracy'])

keras_model.fit(X_train,
          y_train_onehot,
          epochs=20,
          batch_size=128,
          class_weight={0: 100, 1: 1, 2: 100})
score = keras_model.evaluate(X_test, y_test_onehot, batch_size=128)

Using TensorFlow backend.


Epoch 1/20
976/976 [==============================] - 0s 269us/step - loss: 6.5065 - acc: 0.2049
Epoch 2/20
976/976 [==============================] - 0s 37us/step - loss: 10.3563 - acc: 0.3105
Epoch 3/20
976/976 [==============================] - 0s 32us/step - loss: 10.9173 - acc: 0.2500
Epoch 4/20
976/976 [==============================] - 0s 39us/step - loss: 10.1285 - acc: 0.0922
Epoch 5/20
976/976 [==============================] - 0s 49us/step - loss: 4.5143 - acc: 0.0205
Epoch 6/20
976/976 [==============================] - 0s 50us/step - loss: 4.3351 - acc: 0.0215
Epoch 7/20
976/976 [==============================] - 0s 53us/step - loss: 4.7658 - acc: 0.0225
Epoch 8/20
976/976 [==============================] - 0s 42us/step - loss: 4.1169 - acc: 0.0205
Epoch 9/20
976/976 [==============================] - 0s 38us/step - loss: 4.1942 - acc: 0.0205
Epoch 10/20
976/976 [==============================] - 0s 44us/step - loss: 4.0015 - acc: 0.0256
Epoch 11/20
976/976 [==============

In [0]:
print('Accuracy:', score[1])

Accuracy: 0.03265306142595958


In [0]:
from sklearn.metrics import classification_report

y_pred = keras_model.predict_classes(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         3
           0       1.00      0.01      0.02       238
           1       0.00      0.00      0.00         4
           2       0.00      0.00      0.00         0

    accuracy                           0.01       245
   macro avg       0.25      0.00      0.00       245
weighted avg       0.97      0.01      0.02       245



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
